__萌新瑟瑟发抖，第一次搞NLP的比赛。开始的几天查资料搞出了一个简单的Bert版本，分数0.66左右，调了一阵没提升就没再搞这个版本了，当然那时候的数据构造和文本清洗都非常粗糙，后面也没再尝试一下，转头去开始学习一些经典的NN模型了，相关代码也在下一篇里。__

阿里的环境跑不了，我是在kaggle跑的

In [4]:
import warnings
warnings.filterwarnings("ignore")
import os,re
import pandas as pd
import numpy as np

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
from tensorflow.keras import Model, Input, backend as K
import transformers #huggingface transformers library

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

Num GPUs Available:  0


In [5]:
# 加载文本数据
TAIL_VALUES = 10

sel_data = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata//preliminary_sel_log_dataset.csv')
sel_data2 = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata//preliminary_sel_log_dataset_a.csv')
sel_data = pd.concat([sel_data,sel_data2]).sort_values(by=['sn', 'time'], ignore_index=True)
sel_data['time'] = pd.to_datetime(sel_data['time'])
sel_data['week'] = sel_data['time'].dt.isocalendar().week
sel_data = sel_data.groupby(['sn']).tail(TAIL_VALUES)

sel_data2.sort_values(by=['sn', 'time'], ignore_index=True, inplace=True)
sel_data2['time'] = pd.to_datetime(sel_data2['time'])
sel_data2['week'] = sel_data2['time'].dt.isocalendar().week
print(sel_data.shape)
print(sel_data2.shape)

# 加载标签数据
train_label = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata//preliminary_train_label_dataset.csv')
train_label2 = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata//preliminary_train_label_dataset_s.csv')
train_label = pd.concat([train_label, train_label2]).sort_values(by=['sn', 'fault_time'],ignore_index=True)
train_label.drop_duplicates(subset=['sn','fault_time','label'],inplace=True)
train_label['fault_time'] = pd.to_datetime(train_label['fault_time'])
train_label['week'] = train_label['fault_time'].dt.isocalendar().week
print(train_label.shape)

# 加载测试数据
test_data = pd.read_csv('D:/AI/TianCHI/competition_baselines-master/competitions/tianchi_aiops2022/Rawdata//preliminary_submit_dataset_a.csv')
test_data['fault_time'] = pd.to_datetime(test_data['fault_time'])
test_data = test_data.sort_values(by=['sn', 'fault_time'],ignore_index=True)
test_data['week'] = test_data['fault_time'].dt.isocalendar().week
print(test_data.shape)

(103505, 5)
(10991, 5)
(16604, 4)
(3011, 3)


In [6]:
%%time

# 构造训练集
train = pd.merge(sel_data, train_label,on=['sn','week'],how='left')
train = train.query('time <= fault_time')
train.drop_duplicates(subset=['sn','time','msg','fault_time'],inplace=True)
train['text'] = train.groupby(['sn','fault_time','week'])['msg'].transform(lambda x : '.'.join(x))
train['text'] = train['text'].str.lower()
train.drop(['msg','time','week'],axis=1,inplace=True)

train.drop_duplicates(subset=['sn','fault_time','text'],inplace=True)
train.reset_index(inplace=True,drop = True)
train['label'] = train['label'].astype(int)
print(train.label.value_counts())

# 构造测试集
test = pd.merge(test_data, sel_data2,on=['sn','week'],how="left")
test = test.query('time <= fault_time')
test.drop_duplicates(subset=['sn','time','msg','fault_time'],inplace=True)
test['text'] = test.groupby(['sn','fault_time','week'])['msg'].transform(lambda x : '.'.join(x))
test['text'] = test['text'].str.lower()
test.drop(['msg','time','week'],axis=1,inplace=True)
test.drop_duplicates(subset=['sn','fault_time','text'],ignore_index=True,inplace=True)

# 补充测试集
sn_list = ['05aceb8f3b0f', '0e074cf16c7d', '287cfc92991d', '3845fe1dbef9', '82a187bcfa02', 'b64f4cefbbc6', 'd7ca15888043']
supplement_test = test_data.query("sn in @sn_list").copy()
supplement_sel_data = sel_data2.query("sn in @sn_list").copy()

supplement_test = pd.merge(supplement_test, supplement_sel_data,on=['sn'],how="left")
supplement_test['text'] = supplement_test.groupby(['sn','fault_time'])['msg'].transform(lambda x : '.'.join(x))
supplement_test['text'] = supplement_test['text'].str.lower()
supplement_test.drop(['msg','time','week_x','week_y'], axis=1,inplace=True)
supplement_test.drop_duplicates(subset=['sn','fault_time','text'],ignore_index=True,inplace=True)

test = pd.concat([test, supplement_test]).sort_values(by=['sn', 'fault_time'],ignore_index=True)
test.reset_index(inplace=True,drop = True)
print('\n',test.shape)

2    8201
1    3234
3    2310
0    1392
Name: label, dtype: int64

 (3011, 4)
Wall time: 2.28 s


In [4]:
print(train)

                 sn server_model          fault_time  label  \
0      SERVER_10001         SM57 2020-05-01 10:04:00      1   
1      SERVER_10008         SM53 2020-02-25 16:12:00      1   
2      SERVER_10008         SM53 2020-03-11 18:04:00      2   
3      SERVER_10009         SM53 2020-05-08 16:37:00      3   
4      SERVER_10012         SM53 2020-07-13 03:32:00      3   
...             ...          ...                 ...    ...   
15132   SERVER_9991         SM56 2020-08-04 22:49:00      2   
15133   SERVER_9991         SM56 2020-10-07 18:42:00      2   
15134   SERVER_9993         SM57 2020-05-14 23:50:00      2   
15135   SERVER_9998         SM57 2020-05-29 11:25:00      2   
15136   SERVER_9999         SM57 2020-10-13 02:57:00      2   

                                                    text  
0       processor cpu1_status | ierr | asserted. proc...  
1       memory dimm050_stat | uncorrectable ecc | ass...  
2       system boot initiated bios_boot_up | initiate...  
3      

In [7]:
print(test)

                sn          fault_time server_model  \
0     000d33b21436 2020-09-02 16:42:54         SM40   
1     005c5a9218ba 2020-06-28 19:05:16         SM99   
2     0079283bde6e 2020-04-26 21:32:44         SM14   
3     007bdf23b62f 2020-06-16 18:40:39         SM93   
4     00a577a8e54f 2020-04-07 07:16:55         SM13   
...            ...                 ...          ...   
3006  ffbf46b4af21 2019-12-28 20:10:01         SM35   
3007  ffc229b6cd9a 2020-06-27 02:39:08         SM49   
3008  ffd44698a52b 2020-01-21 15:46:56         SM66   
3009  fff73a9e5bd5 2020-03-01 22:43:43         SM92   
3010  fffd22fffe19 2020-01-21 19:22:56         SM16   

                                                   text  
0      system boot initiated bios_boot_up | initiate...  
1      memory memory_status | correctable ecc | asse...  
2      power supply psu1_supply | failure detected |...  
3      memory #0xe2 | correctable ecc | asserted. me...  
4      memory mem_chg1_status | correctable ecc |

In [8]:
print(supplement_test)

             sn          fault_time server_model  \
0  05aceb8f3b0f 2020-03-16 00:41:54         SM23   
1  0e074cf16c7d 2020-11-16 00:12:54         SM35   
2  287cfc92991d 2020-07-06 00:42:55         SM23   
3  3845fe1dbef9 2020-10-26 00:07:54         SM35   
4  82a187bcfa02 2020-04-06 00:26:30         SM16   
5  b64f4cefbbc6 2020-11-23 03:03:53         SM26   
6  d7ca15888043 2020-07-13 01:53:54         SM17   

                                                text  
0   unknown #0x17 |  | asserted. processor cpu_ca...  
1   processor cpu0_status | configuration error |...  
2   unknown #0x17 |  | asserted. processor cpu_ca...  
3   processor cpu1_status | configuration error |...  
4   system acpi power state acpi_pwr_status | s4/...  
5   processor cpu0_status |  | asserted. processo...  
6   processor cpu_caterr | state asserted | asserted  


In [9]:
# train_two = train.query("label == 2").copy()
# train_two = train_two[-4000:].copy()
# train_other = train.query("label != 2").copy()
# train = pd.concat([train_other,train_two])
# train.reset_index(inplace=True,drop = True)
# print(train.label.value_counts())

In [10]:
def strip_all_entities(text):
    text = re.sub('锛�','', text)
    text = re.sub('锟絋',' ', text)
    text = re.sub('锟�','', text)
    text = re.sub('\|',',', text)
    text = re.sub('s4/s5:', 'ss_one_state', text)
    text = re.sub('s0/g0:', 'sg_one_state', text)
    text = re.sub('s5/g2:', 'sg_two_state', text)
    text = re.sub('aa17.{22}','', text)
    text = re.sub('000000','a Special tags', text)
    text = re.sub('\d{4}\w\d\w\d{5}','Asserted oem record', text)
    return text
train['text'] = train['text'].apply(lambda x: strip_all_entities(x))
test['text'] = test['text'].apply(lambda x: strip_all_entities(x))

In [11]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus( #batch_encode_plus
        texts, 
        return_attention_mask=False, 
        return_token_type_ids=False,
        padding = 'max_length',
        max_length = maxlen,
        truncation = True,
#         return_tensors = 'tf'
    )
    
    return np.array(enc_di['input_ids'])

In [12]:
model_path = "D:/AI/TianCHI/tianchi_aiops2022/pretrain/bert-large-uncased/"  # bert-base-uncased、bert-large-uncased
#model_path = "bert-base-uncased" 
tokenizer = transformers.BertTokenizer.from_pretrained(model_path)

In [14]:
#tokenizer = BertTokenizer.from_pretrained('C:\\Users\\lgy\\Desktop\\fsdownload\\bert-base-uncased')
print(tokenizer.mask_token) # [MASK]
print(tokenizer.convert_tokens_to_ids('a')) # 1037
print(tokenizer.convert_ids_to_tokens(1037)) # a
 
string = "test batch encode plus"
strings = [string,string]
tokens = tokenizer.tokenize(string)
print(tokens)#['test', 'batch', 'en', '##code', 'plus']
out = tokenizer.batch_encode_plus(strings,max_length=10,padding='max_length',truncation='longest_first')#长的截，短的补
print(out)

[MASK]
1037
a
['test', 'batch', 'en', '##code', 'plus']


AttributeError: 'BertTokenizer' object has no attribute 'batch_encode_plus'

In [12]:
X = train['text']
y = train['label']
X_train,X_test ,y_train,y_test = train_test_split(X, y, random_state = 2020, test_size = 0.2)
max_len = 128
Xtrain_encoded = regular_encode(X_train, tokenizer, maxlen=max_len)
ytrain_encoded = tf.keras.utils.to_categorical(y_train, num_classes=4,dtype = 'int32')
Xtest_encoded = regular_encode(X_test, tokenizer, maxlen=max_len)
ytest_encoded = tf.keras.utils.to_categorical(y_test, num_classes=4,dtype = 'int32')

Xtest_sub = regular_encode(test['text'], tokenizer, maxlen=max_len)

In [13]:
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [14]:
def build_model(transformer, loss='categorical_crossentropy', max_len=512):
    input_word_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    #adding dropout layer
    x = tf.keras.layers.Dropout(0.3)(cls_token)
    #using a dense layer of 40 neurons as the number of unique categories is 40. 
    out = tf.keras.layers.Dense(4, activation='softmax')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=out)
    #using categorical crossentropy as the loss as it is a multi-class classification problem
    model.compile(tf.keras.optimizers.Adam(lr=3e-5), loss=loss, metrics=['accuracy', f1])
    return model

In [20]:
# with strategy.scope():
transformer_layer = transformers.TFAutoModel.from_pretrained(model_path)
#transformer_layer = transformers.from_pretrained(model_path)
model = build_model(transformer_layer, max_len=max_len)
model.summary()

ImportError: 
TFAutoModel requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.


In [ ]:
#creating the training and testing dataset.
BATCH_SIZE = 64
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((Xtrain_encoded, ytrain_encoded))
    .repeat()
    .shuffle(10000)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
    )

val_dataset = (
    tf.data.Dataset
    .from_tensor_slices((Xtest_encoded,ytest_encoded))
    .shuffle(10000)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
    )

test_sub_dataset = (
    tf.data.Dataset
    .from_tensor_slices(Xtest_sub)
    .batch(BATCH_SIZE)
    )

In [ ]:
n_steps = Xtrain_encoded.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=val_dataset,
    epochs=18
)

In [ ]:
preds = model.predict(test_sub_dataset,verbose = 1)
pred_classes = np.argmax(preds, axis = 1)

In [ ]:
sub = test[['sn', 'fault_time']].copy()
sub['label'] = pred_classes
display(sub.head())
sub['label'].value_counts()

In [ ]:
sub.to_csv('bert_uncase_old.csv', index=False)